In [2]:
import requests

def get_order_book(symbol, limit=100):
    url = "https://api.binance.com/api/v3/depth"
    params = {'symbol': symbol, 'limit': limit}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

order_book = get_order_book('BTCUSDT', 100)
if order_book:
    print(order_book)


{'lastUpdateId': 40127080498, 'bids': [['35161.92000000', '0.45408000'], ['35161.75000000', '0.00409000'], ['35161.49000000', '0.00070000'], ['35161.11000000', '0.01738000'], ['35161.10000000', '0.02840000'], ['35161.05000000', '0.00070000'], ['35160.98000000', '0.01023000'], ['35160.71000000', '0.00029000'], ['35160.61000000', '0.00070000'], ['35160.43000000', '0.00938000'], ['35160.40000000', '0.00060000'], ['35160.37000000', '0.00050000'], ['35160.17000000', '0.00070000'], ['35160.00000000', '0.00170000'], ['35159.85000000', '0.00020000'], ['35159.78000000', '0.06020000'], ['35159.76000000', '0.00568000'], ['35159.73000000', '0.00070000'], ['35159.68000000', '0.00134000'], ['35159.64000000', '0.02923000'], ['35159.35000000', '0.17055000'], ['35159.29000000', '0.00070000'], ['35159.27000000', '0.17063000'], ['35158.95000000', '0.02923000'], ['35158.93000000', '0.00134000'], ['35158.87000000', '0.00072000'], ['35158.86000000', '0.24686000'], ['35158.85000000', '0.00070000'], ['35158.4

In [2]:
!pip install websocket-client


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import websocket
import json

def on_open(ws):
    print("Opened connection")

def on_message(ws, message):
    data = json.loads(message)
    print("Received data:")
    print(data)

def on_error(ws, error):
    print("Error:", error)

def on_close(ws):
    print("Closed connection")

# Replace 'BTCUSDT' with your desired trading pair
symbol = 'BTCUSDT'
stream = f"wss://stream.binance.com:9443/ws/{symbol.lower()}@depth"

# Create and run the WebSocket
ws = websocket.WebSocketApp(stream,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()

Opened connection
Received data:
{'e': 'depthUpdate', 'E': 1699271766201, 's': 'BTCUSDT', 'U': 40126785847, 'u': 40126786063, 'b': [['35229.99000000', '6.21725000'], ['35229.98000000', '0.05967000'], ['35229.77000000', '0.76077000'], ['35229.26000000', '0.68594000'], ['35229.19000000', '0.00000000'], ['35229.00000000', '0.13783000'], ['35228.98000000', '0.11546000'], ['35228.97000000', '0.00000000'], ['35228.65000000', '0.00000000'], ['35228.15000000', '0.04446000'], ['35228.14000000', '0.05217000'], ['35227.95000000', '0.02923000'], ['35227.48000000', '0.00000000'], ['35227.38000000', '0.00000000'], ['35227.14000000', '0.00072000'], ['35226.88000000', '0.00000000'], ['35226.82000000', '0.00104000'], ['35226.81000000', '0.01285000'], ['35226.33000000', '0.00000000'], ['35226.15000000', '0.00000000'], ['35225.51000000', '0.00000000'], ['35225.38000000', '0.00000000'], ['35225.30000000', '0.00000000'], ['35225.20000000', '0.32291000'], ['35225.14000000', '0.56748000'], ['35225.03000000',

True

In [3]:
import websocket
import json
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize Plotly figure
fig = make_subplots(rows=1, cols=1)
fig.update_layout(title='Real-Time Bid Ask Volume')

# Initialize DataFrame to hold the order book data
order_book = pd.DataFrame(columns=['price', 'volume', 'type'])

def on_open(ws):
    print("Opened connection")

def on_message(ws, message):
    global order_book
    global fig  # Ensure to use the global figure

    data = json.loads(message)

    # Convert updates to DataFrame directly
    bid_updates = pd.DataFrame(data['b'], columns=['price', 'volume']).assign(type='bid')
    ask_updates = pd.DataFrame(data['a'], columns=['price', 'volume']).assign(type='ask')

    # Combine bid and ask updates
    updates = pd.concat([bid_updates, ask_updates])

    # Convert price and volume to numeric types and filter out zero volume entries
    updates['price'] = pd.to_numeric(updates['price'])
    updates['volume'] = pd.to_numeric(updates['volume'])
    updates = updates[updates['volume'] > 0]

    # Concatenate new updates with the existing order book and remove duplicates
    order_book = pd.concat([order_book, updates]).drop_duplicates(subset=['price', 'type'], keep='last')

    # Assume last price is the mid price between the highest bid and the lowest ask
    mid_price = (order_book[order_book['type'] == 'bid']['price'].max() +
                 order_book[order_book['type'] == 'ask']['price'].min()) / 2

    # Filter orders within 1% of mid-price
    one_percent_range = mid_price * 0.01
    filtered_book = order_book[(order_book['price'] >= mid_price - one_percent_range) &
                               (order_book['price'] <= mid_price + one_percent_range)]

    # Calculate total bid and ask volume within 1% of mid-price
    total_bid_volume = filtered_book[filtered_book['type'] == 'bid']['volume'].sum()
    total_ask_volume = filtered_book[filtered_book['type'] == 'ask']['volume'].sum()

    # Update the Plotly figure
    fig.data = []  # Clear existing data
    fig.add_trace(go.Bar(x=['Bids', 'Asks'], y=[total_bid_volume, total_ask_volume]))
    fig.show(renderer="browser")  # Update the figure to be shown in a browser


def on_error(ws, error):
    print("Error:", error)

def on_close(ws):
    print("Closed connection")

# Replace 'BTCUSDT' with your desired trading pair
symbol = 'BTCUSDT'
stream = f"wss://stream.binance.com:9443/ws/{symbol.lower()}@depth"

# Create and run the WebSocket
ws = websocket.WebSocketApp(stream,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()


Opened connection


/tmp/ipykernel_11016/1736397560.py:36: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Error: 
Error: on_close() takes 1 positional argument but 3 were given


True

In [4]:
import requests

# Function to get historical order book data from Binance
def get_historical_order_book(symbol, limit=1000):
    url = f'https://api.binance.com/api/v3/depth?symbol={symbol}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    return data

# Replace 'BTCUSDT' with your desired trading pair
symbol = 'BTCUSDT'
historical_order_book = get_historical_order_book(symbol)

# Convert to DataFrame
bids_df = pd.DataFrame(historical_order_book['bids'], columns=['price', 'volume']).assign(type='bid')
asks_df = pd.DataFrame(historical_order_book['asks'], columns=['price', 'volume']).assign(type='ask')
historical_order_book_df = pd.concat([bids_df, asks_df])

# Convert price and volume to numeric types
historical_order_book_df['price'] = pd.to_numeric(historical_order_book_df['price'])
historical_order_book_df['volume'] = pd.to_numeric(historical_order_book_df['volume'])

# Now you can concatenate this historical data with your real-time updates in the `on_message` function


In [7]:
get_historical_order_book(symbol)

{'lastUpdateId': 40127099000,
 'bids': [['35131.10000000', '0.22736000'],
  ['35130.85000000', '0.00038000'],
  ['35130.76000000', '0.00044000'],
  ['35130.69000000', '0.00638000'],
  ['35130.43000000', '0.00158000'],
  ['35130.25000000', '0.00111000'],
  ['35130.08000000', '0.00098000'],
  ['35130.06000000', '0.00072000'],
  ['35130.05000000', '0.00048000'],
  ['35130.00000000', '0.00103000'],
  ['35129.87000000', '0.00017000'],
  ['35129.81000000', '0.00070000'],
  ['35129.80000000', '0.00625000'],
  ['35129.75000000', '0.00020000'],
  ['35129.68000000', '0.00134000'],
  ['35129.58000000', '0.02850000'],
  ['35129.37000000', '0.00070000'],
  ['35129.16000000', '0.00360000'],
  ['35129.13000000', '0.00113000'],
  ['35129.01000000', '0.00021000'],
  ['35129.00000000', '0.00603000'],
  ['35128.96000000', '0.00060000'],
  ['35128.93000000', '0.00204000'],
  ['35128.70000000', '0.00089000'],
  ['35128.60000000', '0.00040000'],
  ['35128.57000000', '0.01778000'],
  ['35128.50000000', '0.00

In [5]:
historical_order_book_df

,price,volume,type
0,35150.68,1.03830,bid
1,35150.60,0.00015,bid
2,35150.58,0.00033,bid
3,35150.56,0.02475,bid
4,35150.49,0.00070,bid
...,...,...,...
995,35280.61,0.00162,ask
996,35280.65,0.00077,ask
997,35280.71,0.00089,ask
998,35280.79,0.00040,ask
